# Colour map

to make the episode descriptions as generic as possible, we'll map each character in each season to a generic colour label, allowing for interchangability. 

steps involved:
1. scrape characters / power ranger names
2. extract colours from the power ranger names
3. assign colours to some of the weird ones as I deffo saw some power rangers called like 'phantom' which isnt a colour and isnt helpful
4. probs also take characters first names as an extra column too.

In [10]:
import pandas as pd
from colour import Color
from typing import Optional

so lets get started by readin in the scraped data and seeing where we stand

In [18]:
df = pd.read_excel('data/colour_list.xlsx')
df.head()

,Color,Role,Actor,season
0,Red Ranger,Jason Lee Scott,Austin St. John,mighty morphin
1,Black Ranger,Zack Taylor,Walter Jones,mighty morphin
2,Blue Ranger,Billy Cranston,David Yost,mighty morphin
3,Yellow Ranger,Trini Kwan,Thuy Trang,mighty morphin
4,Pink Ranger,Kimberly Ann Hart,Amy Jo Johnson,mighty morphin


we'll start with some quick housekeeping, namely some decapitalising, prehaps a column rename or two...

In [19]:
df.columns = [column.lower() for column in df.columns]
df.rename(columns={'color': 'raw_colour'}, inplace=True)

df['raw_colour'] = df['raw_colour'].str.lower()
df['character'] = df['role'].str.split().str[0]

df.head()

,raw_colour,role,actor,season,character
0,red ranger,Jason Lee Scott,Austin St. John,mighty morphin,Jason
1,black ranger,Zack Taylor,Walter Jones,mighty morphin,Zack
2,blue ranger,Billy Cranston,David Yost,mighty morphin,Billy
3,yellow ranger,Trini Kwan,Thuy Trang,mighty morphin,Trini
4,pink ranger,Kimberly Ann Hart,Amy Jo Johnson,mighty morphin,Kimberly


fab. Next I'll extract the colours where they exist...

In [24]:
def check_color(color):
    try:
        Color(color)
        return True
    except ValueError:
        return False
    

def find_colour_in_text(input_text: str) -> Optional[str]:
    """Extract the colour from a given string
    
    Parameters
    ----------
    input_text : String ideally containing some colour.
    
    Returns
    -------
    str
        The colour contained in input_text, else None
    """
    colour_list = [i for i in input_text.split(' ') if check_color(i)]
    
    if len(colour_list) > 0:
        return colour_list[0]
    else:
        return input_text.replace(' ranger', '')  # mostly these are 'quantumn ranger', or 'deka ranger'.

df['colour'] = df['raw_colour'].apply(find_colour_in_text)
df['colour_ranger'] = df['colour'] + ' ranger'
df.head()

,raw_colour,role,actor,season,character,colour,colour_ranger
0,red ranger,Jason Lee Scott,Austin St. John,mighty morphin,Jason,red,red ranger
1,black ranger,Zack Taylor,Walter Jones,mighty morphin,Zack,black,black ranger
2,blue ranger,Billy Cranston,David Yost,mighty morphin,Billy,blue,blue ranger
3,yellow ranger,Trini Kwan,Thuy Trang,mighty morphin,Trini,yellow,yellow ranger
4,pink ranger,Kimberly Ann Hart,Amy Jo Johnson,mighty morphin,Kimberly,pink,pink ranger


fabulous. I think we are now ready to progress to scraping episode synopsis' for every episode, whereafter we can hopefully use this data to replace any mentions of characters / colours with the simplified versions. its highly likely we'll need to come back and do some further tweaking later but for now this will work :)

In [25]:
df.to_csv('data/colours_cleaned', index=False)